In [1]:
import os

import pandas as pd
from tqdm import tqdm

DATA_DIR = "../data/public"

table_names = [f"main{i}" for i in range(1, 4)]
table_paths = [os.path.join(DATA_DIR, f"{table_name}.csv") for table_name in table_names]
tables = [pd.read_csv(table_path, nrows=1000) for table_path in tqdm(table_paths)]
len(tables)

100%|██████████| 3/3 [00:00<00:00, 179.45it/s]


3

In [2]:
tables[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   uid        1000 non-null   object
 1   full_name  1000 non-null   object
 2   email      1000 non-null   object
 3   address    1000 non-null   object
 4   sex        1000 non-null   object
 5   birthdate  1000 non-null   object
 6   phone      1000 non-null   object
dtypes: object(7)
memory usage: 54.8+ KB


In [3]:
tables[1].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   uid          1000 non-null   object
 1   first_name   1000 non-null   object
 2   middle_name  1000 non-null   object
 3   last_name    1000 non-null   object
 4   birthdate    1000 non-null   object
 5   phone        1000 non-null   object
 6   address      1000 non-null   object
dtypes: object(7)
memory usage: 54.8+ KB


In [4]:
tables[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   uid        1000 non-null   object
 1   name       1000 non-null   object
 2   email      1000 non-null   object
 3   birthdate  1000 non-null   object
 4   sex        1000 non-null   object
dtypes: object(5)
memory usage: 39.2+ KB


In [5]:
from app.custom_dataframe import CustomDataFrame

df = CustomDataFrame()
df.head()

,uid,first_name,middle_name,last_name,birthdate,sex,phone,phone2,address,email,email2,email3


In [6]:
duplicates = []

for table_num, table in enumerate(tables):
    for index in tqdm(range(len(table))):
        row = table.iloc[index]
        response = df.register_row(row)
        if len(duplicates) != 0:
            print(table_num, index)

  5%|▍         | 47/1000 [00:00<00:01, 705.57it/s]


KeyError: "['caae5eb5-25f5-45e9-94c4-5d590403215f'] not found in axis"

In [ ]:
result_df = pd.DataFrame(columns=["uid", "uid1", "uid2", "uid3"])